파이썬 머신러닝 완벽 가이드 ch9. 8 pg.607~624
## 08. 파이썬 추천 시스템 패키지 - Surprise

### Surprise 패키지 소개

- 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지
- 다양한 추천 알고리즘을 쉽게 적용해 추천 시스템을 구축할 수 있음
- 핵심 API가 사이킷런과 유사한 이름으로 작성됨

In [1]:
!pip install surprise

### Surprise를 이용한 추천 시스템 구축
- 예제 - 추천 데이터를 학습용과 테스트용 데이터 세트로 분리한 뒤 SVD 행렬 분해를 통한 잠재 요인 협업 필터링 수행

In [2]:
from surprise import SVD
from surprise import Dataset #Surprise에서 데이터 로딩은 Dataset 클래스를 이용해서만 가능
from surprise import accuracy
from surprise.model_selection import train_test_split

- 주요 데이터가 사용자 아이디, 아이템 아이디, 평점 데이터가row 레벨 형태로 돼있는 포맷의 데이터만 처리함

In [3]:
#MovieLens 사이트에서 제공하는 과거 버전의 데이터세트를 가져오는 API 제공
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

In [4]:
#SVD로 잠재 요인 협업 필터링 수행
algo = SVD()  #알고리즘 객체 생성
algo.fit(trainset)

- 추천 예측 메서드
  - test() : 사용자-아이템 평점 데이터 세트 전체에 대해서 추천 예측, 입력된 데이터 세트에 대해 추천 데이터 세트 만들어줌
  - predict()  : 개별 사용자와 영화에 대한 추천 평점 반환
  

In [5]:
#test()
predictions = algo.test(testset)
print('prediction type : ', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type :  <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.758398559178154, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8908669282728736, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.940938399891338, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.4336761610289543, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.5592241327189638, details={'was_impossible': False})]

    - SVD 알고리즘 객체의 test() 메서드의 호출 결과는 파이썬 리스트, 크기는 입력 인자 데이터 세트의 크기와 같은 25000개로 Prediction 객체를 가짐
        - Prediction 객체 : Surprise 패키지에서 제공하는 데이터 타입, 추천 예측 평점 데이터를 튜플 형태로 가짐
            - was_impossible이 True 면 예측값ㅇ르 생성할 수 없는 데이터 의미

In [6]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.758398559178154),
 ('882', '291', 3.8908669282728736),
 ('535', '507', 3.940938399891338)]

In [7]:
#predict()

#사용자 아이디, 아이템 아이디는 문자열로 입력해야함
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.22   {'was_impossible': False}


    - 개별 사용자와 아이템 정보를 입력하면 추천 예측 평점을 est로 반환
    - test() 메서드는 입력 데이터 세트의 모든 사용자와 아이템 아이디에 대해서 predict() 반복적으로 수행한 결과라고 이해

In [8]:
#accuracy - rmse, mse
accuracy.rmse(predictions)

RMSE: 0.9505


0.9504616550158783

### Surprise 주요 모듈 소개
#### Dataset
- load_builtin(name='ml-100k') : 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터를 내려받음, ml-100k, ml-1M 내려 받을 수 있음
- load_from_file(file_path, reader) : OS 파일에서 데이터를 로딩할 때 사용, 파일명, 파일포맷 지정
- load_from_df(df, reader) : 판다스의 DataFrame에서 데이터 로딩, DataFrame 역시 반드시 사용자 아이디, 아이템 아이디, 평점 순으로 칼럼이 정해져 있어야 함, DataFrame 객체, 파일 포맷 지정
#### OS 파일 데이터를 Surprise 데이터 세트로 로딩
- 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있으면 안 됨


In [14]:
import pandas as pd

ratings = pd.read_csv('/Users/sooyeon/Downloads/archive (4)/ratings.csv')
#ratings_noh.csv 파일로 언로드시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('/Users/sooyeon/Downloads/archive (4)/ratings_noh.csv', index=False, header=False)

- load_from_file() 적용하기 전에 Reader 클래스를 이용해 데이터 파일의 파싱 포맷 정의해야 함
    - 생성자에 각 필드의 칼럼명과 칼럼 분리문자, 최소~최대 평점 입력해서 객체 생성

In [16]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data = Dataset.load_from_file('/Users/sooyeon/Downloads/archive (4)/ratings_noh.csv', reader=reader)

In [17]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

#학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### 판다스 DataFrame에서 Surprise 데이터 세트로 로딩
- DataFrame 역시 사용자 아이디, 아이템 아이디, 평점 칼럼 순서 지켜야 함

In [18]:
ratings = pd.read_csv('/Users/sooyeon/Downloads/archive (4)/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

#ratings DataFrame 에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### Surprise 추천 알고리즘 클래스
- KNNBasic : 최근접 이웃 협업 필터링 위한 KNN 알고리즘
- BaeslineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘
- SVD : 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
    - 비용 함수 : 사용자 baseline 편향성을 감안한 평점 예측에 regularization 적용한 것
        - n_factors : 잠재 요인 K의 개수, 디폴트 100, 커질수록 정확도가 높아질 수 있으나 과적합 문제가 발생할 수 있음
        - n_epochs : SGD 수행 시 반복 횟수, 디폴트 20
        - biased(bool) : 베이스라인 사용자 편향 적용 여부, 디폴트 True
    


### 베이스라인 평점
- 각 개인이 평점을 부여하는 성형을 반영해 평점을 계산하는 방식
- 보통 $ 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 $ 로 계산
    - 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
    - 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
    - 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점

### 교차 검증과 하이퍼 파라미터 튜닝
- 사이킷런과 유사한 cross_validate()와 GridSearchCV 클래스 제공

In [20]:
#cross_validate
from surprise.model_selection import cross_validate

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8738  0.8691  0.8664  0.8755  0.8780  0.8726  0.0042  
MAE (testset)     0.6721  0.6697  0.6679  0.6719  0.6736  0.6711  0.0020  
Fit time          0.47    0.51    0.49    0.47    0.49    0.48    0.01    
Test time         0.08    0.14    0.08    0.08    0.09    0.09    0.02    


{'test_rmse': array([0.87382872, 0.86911587, 0.86643131, 0.87553848, 0.87798113]),
 'test_mae': array([0.67209898, 0.66973409, 0.6679362 , 0.67188237, 0.67363159]),
 'fit_time': (0.4722902774810791,
  0.506854772567749,
  0.48555779457092285,
  0.4662809371948242,
  0.49326395988464355),
 'test_time': (0.08236384391784668,
  0.13798308372497559,
  0.08178019523620605,
  0.0829932689666748,
  0.08505797386169434)}

    - 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치 함께 보여줌

In [22]:
#GridSearchCV
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs':[20, 40, 60], 'n_factors':[50,100,200]}

#CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
gs.fit(data)

#최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8781396160762761
{'n_epochs': 20, 'n_factors': 50}


### Surprise를 이용한 개인화 영화 추천 시스템 구축
- 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 개인 취향에 가장 적절한 영화 추천
- ratings.csv 데이터 전체를 학습 데이터로 사용
    - Surprise는 데이터 세트를 train_test_split()을 이용해 내부에서 사용한 TrainSet 클래스 객체로 변환하지 않으면 fit()을 통해 학습할 수가 없음
    - DatasetAutoFolds 클래스 이용

In [23]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
#DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='/Users/sooyeon/Downloads/archive (4)/ratings_noh.csv', reader=reader)

#전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [ ]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [24]:
#userId = 9

#영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('/Users/sooyeon/Downloads/archive (4)/movies.csv')

#userId=9 의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count()==0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
    
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [25]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.50   {'was_impossible': False}


In [26]:
#사용자가 평점을 매기지 않은 전체 영화 추출
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    
    #모든 영화의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()
    
    #모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수 : ', len(unseen_movies), 
          '전체 영화 수 :', len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수 :  9696 전체 영화 수 : 9742


In [27]:
#추천 대상 영화 모두를 대상으로 추천 알고리즘 객체의 predict() 메소드를 호출하고 그 결과인 Prediction 객체를 리스트 객체로 저장
#저장된 리스트의 Prediction 객체를 예측 평점이 높은 순으로 다시 정렬한 뒤 top-n개의 영화 아이디, 영화 제목, 예측 평점 정보 추출해 반환
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    
  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
  # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
  # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]

  # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
  # sortkey_est 함수는 list 객체에서 sort() 함수의 키 값으로 사용되어 정렬 수행
  def sortkey_est(pred):
    return pred.est

  # sortkey_est() 반환값의 내림차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]

  # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title'] 

  top_movie_preds = [(id, title, rating) for id, title, rating in 
                     zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print ('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
  print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수 :  9696 전체 영화 수 : 9742
##### Top-10 추천 영화 리스트 #####
Toy Story (1995) : 3.5032292454350493
Jumanji (1995) : 3.5032292454350493
Grumpier Old Men (1995) : 3.5032292454350493
Waiting to Exhale (1995) : 3.5032292454350493
Father of the Bride Part II (1995) : 3.5032292454350493
Heat (1995) : 3.5032292454350493
Sabrina (1995) : 3.5032292454350493
Tom and Huck (1995) : 3.5032292454350493
Sudden Death (1995) : 3.5032292454350493
GoldenEye (1995) : 3.5032292454350493
